we wanna aggregate all datasets in one clean data set 
### columns are : 
company         : 1-ONCF,2-CTM,3-TAXI,4-Covoiturage

from_lat        : latitude of the origine city

from_lan        : langitude of the origine city

to_lat          : latitude of the destination city

to_lan          : langitude of the destination city

distance        : distance between origin and destination

duration        : traveling duration

price           : travling cost


In [1]:
import numpy as np
import requests
import json
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import re
import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression

In [16]:
def getLocationOfCity(city):
    api_key = '966a766bb88e4cb5a5f78ffd9ec99208'
    req = requests.get('https://api.opencagedata.com/geocode/v1/json?q='+city+'&key='+api_key)
    js = json.loads(req.text)
    rep = None
    for res in js['results']:
        if res['formatted'].split(',')[-1] == " Morocco":
            rep = res['geometry'] 
            break
    return rep

In [17]:
def distance(locationA,locationB):
    R = 6373.0

    lat1 = radians(locationA['lat'])
    lon1 = radians(locationA['lng'])
    lat2 = radians(locationB['lat'])
    lon2 = radians(locationB['lng'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [18]:
oncfDF = pd.read_csv("oncfTrips.csv",delimiter=";",encoding='utf-8')
ctmDF = pd.read_csv("CTMTrips.csv",delimiter=";",encoding='latin-1')
covDF = pd.read_csv("cov.csv",delimiter=",",encoding='latin-1')

In [19]:
cities = list(oncfDF['from'])
cities.extend(list(oncfDF['to']))
cities.extend(list(ctmDF['from']))
cities.extend(list(ctmDF['to']))

In [20]:
cities = list(set(cities))
cities = [city.upper() for city in cities]
cities = list(set(cities))

In [21]:
locations = {}
for city in cities:
    locations[city] = getLocationOfCity(city)

In [22]:
locations['SIDI SLIMANE MEDINA']={'lat': 34.2737827,'lng':-5.9806831}
locations['KELAA  DES  SRAGHNAS']={'lat' :32.0510291,'lng':-7.4206207}
locations['GUELMIMA']={'lat' :31.6844986,'lng':-4.979725}
locations['AEROPORT MED V']={'lat' :33.3699749,'lng':-7.5879118}

In [23]:
def duration(time1,time2):
    FMT = '%H:%M'
    H1 = time1.split(':')[0]
    H2 = time2.split(':')[0]
    date1 = datetime.datetime.strptime(time1,FMT)
    date2 = datetime.datetime.strptime(time2,FMT) 
    if int(H2) < int(H1):
        date2 += timedelta(days=1)
    diff = date2 - date1
    return diff.seconds / 60

In [24]:
tfrom = oncfDF['from']
tto = oncfDF['to']
tdepart = oncfDF['depart_time']
tarrival = oncfDF['arrival_time']
tprice = oncfDF['price']
data = []
# day of the week will be 0 if this trip repeat everyday
for i in range(len(tfrom)):
    data.append([1,locations[tfrom[i].upper()]['lat'],locations[tfrom[i].upper()]['lng'],locations[tto[i].upper()]['lat'],locations[tto[i].upper()]['lng'],
               distance(locations[tfrom[i].upper()],locations[tto[i].upper()]),
               tdepart[i],duration(tdepart[i],tarrival[i]),0,int(tprice[i][:-3])])
    
    

In [25]:
tfrom = ctmDF['from']
tto = ctmDF['to']
tdepart = ctmDF['depart_time']
tarrival = ctmDF['arrival_time']
tprice = ctmDF['price']
tprice.fillna((tprice.mean()), inplace=True)
for i in range(len(tfrom)):
    data.append([2,locations[tfrom[i].upper()]['lat'],locations[tfrom[i].upper()]['lng'],locations[tto[i].upper()]['lat'],locations[tto[i].upper()]['lng'],
               distance(locations[tfrom[i].upper()],locations[tto[i].upper()]),
                tdepart[i],duration(tdepart[i],tarrival[i]),0,int(tprice[i])])

In [30]:
def timeFromDate(x):
    return x.split(' ')[1]

def minute(x):
    return  60*int(x.split(':')[0]) + int(x.split(':')[1])

def dayOfWeek(x):
    date = x.split(' ')[0]
    year, month, day = (x for x in date.split('-'))
    if day[-1] == ',':
        day = day[:-1]
    if len(day) == 4:
        tmp = day
        day = year
        year= tmp
    day, month, year = int(day), int(month), int(year)
    return datetime.date(year, month, day).weekday() + 1

In [31]:
print(dayOfWeek("2019-03-20 09:58:00"))

3


In [32]:
# from_lat,from_lon,to_lat,to_lon,distance,duree,date_depart,prix
tfrom_lat = covDF['from_lat']
tfrom_lon = covDF['from_lon']
tto_lat = covDF['to_lat']
tto_lon = covDF['to_lon']
tprice = covDF['prix']
tdistance = covDF['distance']
tduree = covDF['duree']
tdepart = covDF['date_depart']
for i in range(len(tfrom_lat)):
    data.append([3,tfrom_lat[i],tfrom_lon[i],tto_lat[i],tto_lon[i],tdistance[i],timeFromDate(tdepart[i]),minute(tduree[i]),dayOfWeek(tdepart[i]),tprice[i]])

In [33]:
X = np.array([86.9,52.8,59.7,64.2,41.3,78.2,64.7,141]).reshape(-1,1)
y = np.array([63,44,68,48,39,57,54,97])
duration_model = LinearRegression().fit(X, y)
duration_model.score(X, y)

0.8443326971108013

In [34]:
def durationFromLoc(locationA,locationB):
    d = distance(locationA,locationB)
    return duration_model.predict([[d]])[0]  

In [35]:
durationFromLoc(locations['MEKNES'],locations['FES'])

45.86008621137471

In [36]:
len(data)

654801

In [37]:
X = np.array([202,54,91,119,60.2,67.5,63.9,24.2,78.1,188,28.8,115,44,126,124,154,85.4,25.3,51,112,128,93.8]).reshape(-1,1)
y = np.array([75,25,35,45,28,25,25,8,35,80,12,50,15,60,55,70,35,10,40,60,50,70])
price_model = LinearRegression().fit(X, y)
price_model.score(X, y)

0.8412497123562451

In [38]:
import random
for a in locations.values():
    for b in locations.values():
        if a!=b:
            data.append([4,a['lat'],a['lng'],b['lat'],b['lng'],distance(a,b),timeFromDate(random.choice(tdepart)),durationFromLoc(a,b),dayOfWeek(random.choice(tdepart)),price_model.predict([[distance(a,b)]])[0]])
    

In [39]:
len(data)

661603

In [41]:
pd.DataFrame(data).to_csv('trips.csv',sep=';',header=["company" ,"from_lat" ,"from_lan" ,"to_lat" ,"to_lan" ,"distance" ,"depart","duration","dayofweek","price"])

In [42]:
def f(x):
    x = int(x.split(":")[0])
    if (x > 4) and (x <= 8):
        return 1
    elif (x > 8) and (x <= 12 ):
        return 2
    elif (x > 12) and (x <= 16):
        return 3
    elif (x > 16) and (x <= 20) :
        return 4
    elif (x > 20) and (x <= 24):
        return 5
    elif (x <= 4):
        return 0
def g(x):
    if x == 'M':
        return 1
    return 2

def h(x):
    p = ['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor']
    for a,b in enumerate(p):
        if x  == b:
            return a
    

In [46]:
trips = pd.read_csv('../datasources/trips.csv',delimiter=";",encoding='utf-8')
users = pd.read_csv('../datasources/users.csv',delimiter=";",encoding='utf-8')
trips.head()

,id,company,from_lat,from_lan,to_lat,to_lan,distance,depart,duration,dayofweek,price
0,0,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,07:50,47.0,0,66.0
1,1,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,07:50,90.0,0,50.0
2,2,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,11:50,47.0,0,66.0
3,3,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,11:50,90.0,0,50.0
4,4,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,16:50,47.0,0,66.0


In [44]:
trips["session"] = trips['depart'].apply(f)
trips = trips.drop(axis=1,columns=['depart'])
users['profition'] = users['profition'].apply(h)
users['sex'] = users['sex'].apply(g)

In [50]:
trips['distance'].median()

322.2420004

In [44]:
n= np.random.randint(500,701)
dd = trips.sample(n,random_state=1)
for j in range(1,5):
    tmp = dd.loc[dd['company']==j]
    p = len(tmp)
    for k,l in zip(range(p),tmp['id']):
        print(k,l)

0 4218
1 2484
2 2762
3 13050
4 4917
5 1149
6 2636
7 6423
8 3335
9 5430
10 6070
11 5451
12 4034
0 14166
1 15378
2 16795
3 16051
0 371683
1 210123
2 278799
3 344628
4 141821
5 212778
6 76534
7 569443
8 520961
9 395253
10 140514
11 465653
12 547118
13 218421
14 364883
15 583203
16 297712
17 213859
18 382923
19 398085
20 282776
21 310648
22 284141
23 487125
24 591728
25 171942
26 624001
27 649155
28 106587
29 196197
30 106434
31 114621
32 501310
33 412215
34 308463
35 422406
36 601386
37 374680
38 191155
39 640988
40 525500
41 628287
42 417187
43 99045
44 627584
45 393778
46 389818
47 115132
48 610530
49 318747
50 305328
51 429754
52 318253
53 134206
54 155735
55 365574
56 446619
57 137452
58 231421
59 366949
60 623222
61 564411
62 530890
63 232481
64 294341
65 294138
66 392630
67 580606
68 159056
69 498741
70 245911
71 146616
72 250667
73 443487
74 580489
75 266601
76 647208
77 105729
78 445581
79 588013
80 649237
81 356786
82 418072
83 640286
84 417880
85 543056
86 350495
87 202680
88 46

In [52]:
hist = []
m = len(users)
for i in range(m):
    dd = trips.sample(1000,random_state=1)
    for j in range(1,5):
        tmp = dd.loc[dd['company']==j]
        p = len(tmp)
        if j == 1:
            security = np.full(shape=p,fill_value=1)
            trafic = np.full(shape=p,fill_value=0)
            retard = np.round(np.random.normal(0.3, 0.1, p) * 60)
        else :
            security = np.round(np.random.normal(0.55, 0.1, p) * 1)
            retard = np.round(np.random.normal(0.3, 0.1, p) * 120)
            trafic = []
            for l in tmp['id']:
                if tmp['dayofweek'][l] == 0:
                    trafic.append(random.randint(0,2))
                if tmp['dayofweek'][l] >=5:
                    trafic.append(1)
                else :
                    trafic.append(0)
        acc = []
        for k,l in zip(range(p),tmp['id']):
            if trafic[k] == 0 and tmp['distance'][l] < 320:
                acc.append(0)
            else :
                acc.append(1)       
        for k,l in zip(range(p),tmp['id']):
            hist.append([users['id'][i],tmp['id'][l],trafic[k],security[k],retard[k],acc[k]])          

In [53]:
pd.DataFrame(hist).to_csv('history.csv',sep=';',header=["user_id","trip_id","trafic","security","retard","accident"],index= False)

In [54]:
hist = pd.read_csv('history.csv',delimiter=";",encoding='utf-8')

In [27]:
len(hist)

943000

In [22]:
trips = pd.read_csv('../datasources/trips.csv',delimiter=';')
users = pd.read_csv('../datasources/users.csv',delimiter=';')
history = pd.read_csv('../datasources/history.csv',delimiter=';')
len(history)

943000

In [23]:
def f(x):
    x = int(x.split(":")[0])
    if (x > 4) and (x <= 8):
        return 1
    elif (x > 8) and (x <= 12 ):
        return 2
    elif (x > 12) and (x <= 16):
        return 3
    elif (x > 16) and (x <= 20) :
        return 4
    elif (x > 20) and (x <= 24):
        return 5
    elif (x <= 4):
        return 0
def g(x):
    if x == 'M':
        return 1
    return 2

def h(x):
    p = ['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor']
    for a,b in enumerate(p):
        if x  == b:
            return a

def ff(x):
    if x<30 :
        return 0
    elif x<60:
        return 1
    elif x<90:
        return 2
    else:
        return 3

In [24]:
trips.head()

,id,company,from_lat,from_lan,to_lat,to_lan,distance,depart,duration,dayofweek,price
0,0,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,07:50,47.0,0,66.0
1,1,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,07:50,90.0,0,50.0
2,2,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,11:50,47.0,0,66.0
3,3,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,11:50,90.0,0,50.0
4,4,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,16:50,47.0,0,66.0


In [25]:
trips["session"] = trips['depart'].apply(f)


In [26]:
trips = trips.drop(axis=1,columns=['depart','id'])


In [27]:
users['profition'] = users['profition'].apply(h)


In [28]:
users['sex'] = users['sex'].apply(g)


In [29]:
users = users.drop(axis=1,columns=['id'])
history['retard'] = history['retard'].apply(ff)

In [30]:
trips.head()

,company,from_lat,from_lan,to_lat,to_lan,distance,duration,dayofweek,price,session
0,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,47.0,0,66.0,1
1,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,90.0,0,50.0,1
2,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,47.0,0,66.0,2
3,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,90.0,0,50.0,2
4,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,47.0,0,66.0,3


In [31]:
rating = {}

rating["0000"] =  5
rating["0001"] =  4
rating["0010"] =  5
rating["0011"] =  5
rating["0100"] =  4
rating["0101"] =  3
rating["0110"] =  4
rating["0111"] =  3
rating["0200"] =  3
rating["0201"] =  2
rating["0210"] =  3
rating["0211"] =  2
rating["0300"] =  2
rating["0301"] =  1
rating["0310"] =  2
rating["0311"] =  1
rating["1000"] =  3
rating["1001"] =  2
rating["1010"] =  3
rating["1011"] =  2
rating["1100"] =  2
rating["1101"] =  1 
rating["1110"] =  2
rating["1111"] =  1
rating["1200"] =  1
rating["1201"] =  0
rating["1210"] =  1
rating["1211"] =  0
rating["1300"] =  0
rating["1301"] =  0
rating["1310"] =  0
rating["1311"] =  0

In [32]:
rt = []
for i in range(len(history)):
    try :
        rt.append([history['user_id'][i],history['trip_id'][i],history['accident'][i],history['retard'][i],history['security'][i],history['trafic'][i],rating[str(int(history['accident'][i]))+str(int(history['retard'][i]))+str(int(history['security'][i]))+str(int(history['trafic'][i]))]])
    except :
        print(i,'err')
        pass

2020 err
3020 err
5020 err
5024 err
7022 err
7024 err
7026 err
8022 err
9022 err
9026 err
10020 err
10024 err
11024 err
12020 err
12024 err
13020 err
14020 err
14022 err
14026 err
15020 err
15022 err
15026 err
18020 err
18026 err
19026 err
20020 err
20024 err
21024 err
22024 err
23022 err
23024 err
24022 err
25020 err
25022 err
26020 err
26024 err
29020 err
29022 err
30026 err
34022 err
35024 err
36024 err
36026 err
37022 err
37024 err
38020 err
38024 err
38026 err
39022 err
39026 err
40020 err
40022 err
40024 err
41024 err
42020 err
42022 err
42024 err
43024 err
43026 err
44026 err
45026 err
46024 err
47022 err
47026 err
48026 err
49026 err
50020 err
51022 err
52024 err
53022 err
53026 err
54022 err
55024 err
57020 err
60020 err
60026 err
61022 err
62022 err
62024 err
63022 err
63026 err
64022 err
64026 err
65024 err
65026 err
66020 err
66024 err
67022 err
67024 err
67026 err
68020 err
68022 err
68024 err
69020 err
70024 err
70026 err
71024 err
72022 err
72024 err
72026 err
73022 err


570020 err
570022 err
571020 err
572022 err
572026 err
574024 err
574026 err
575022 err
575024 err
575026 err
577022 err
578020 err
578022 err
580024 err
581024 err
582026 err
583020 err
583022 err
583024 err
584020 err
584026 err
585022 err
585024 err
586026 err
587022 err
587024 err
587026 err
588024 err
588026 err
589022 err
590020 err
590022 err
590024 err
590026 err
592026 err
593024 err
595024 err
595026 err
596022 err
597020 err
597022 err
597024 err
599024 err
600020 err
600024 err
601024 err
601026 err
602026 err
603022 err
604024 err
605020 err
605022 err
605026 err
606022 err
606024 err
606026 err
607020 err
607024 err
608022 err
608024 err
610022 err
610024 err
610026 err
613022 err
614022 err
614026 err
615020 err
615022 err
616024 err
617024 err
618026 err
619024 err
620020 err
622022 err
623020 err
624020 err
624026 err
627026 err
628026 err
630020 err
630024 err
631022 err
632020 err
633024 err
634026 err
636020 err
637024 err
638022 err
638026 err
639020 err
639022 err

In [33]:
X = []
trips = trips.to_numpy()
users = users.to_numpy()
for r in rt:
    X.append(np.concatenate([trips[r[1]],users[r[0]-1],r[2:-1]]))

In [21]:
trips[1]

array([  1.        ,  33.3699749 ,  -7.5879118 ,  34.2296268 ,
        -6.528318  , 136.87394176,  90.        ,   0.        ,
        50.        ,   1.        ])

In [34]:
data = pd.DataFrame(X,columns=['company', 'from_lat', 'from_lan', 'to_lat', 'to_lan', 'distance',
       'duration','dayofweek','price','session','age', 'gender', 'profission','accident','retard','security','trafic'])

In [35]:
data['rating'] = np.array(rt)[:,-1]

In [36]:
len(data)

941742

In [37]:
data.to_csv("dataset.csv",sep=';',index = False)


In [39]:
data.columns

Index(['company', 'from_lat', 'from_lan', 'to_lat', 'to_lan', 'distance',
       'duration', 'dayofweek', 'price', 'session', 'age', 'gender',
       'profission', 'accident', 'retard', 'security', 'trafic', 'rating'],
      dtype='object')